In [1]:
execution_mode = 'restricted'
factor = 1.0
exactDate_mode = 'xor'

# Feature Matrix Generation

In chapter [Goldstandard and Data Preparation](./2_GoldstandardDataPreparation.ipynb), Swissbib's goldstandard data has been processed to form records of pairs of duplicate and pairs of unique records. These records are the starting point for the final feature matrix generation and that is the reason, why the DataFrame was called feature base. As described in [[JudACaps](./A_References.ipynb#judacaps)], the next step will be an attribute-wise comparison of each attribute pair of each record in the original feature base. This comparison will generate similarity values for each attribute pair. It will halve the number of attributes replacing each attribute pair with one value expressing their degree of similarity. The goal of this chapter is a DataFrame with the full and final feature attributes. The values of these feature attributes will be used for training and performance testing of the machine learning models in the chapters to follow.

This chapter introduces similarity metrics for string comparisons. The metrics to be used for calculating its similarity will be decided for each attribute pair of the DataFrame built in the previous chapters.

## Table of Contents

- [Data Takeover](#Data-Takeover)
- [Object Distance and Similarity](#Object-Distance-and-Similarity)
    - [Mathematical Definitions](#Mathematical-Definitions)
    - [Library TextDistance](#Library-TextDistance)
- [Similarity Metrics on Attribute Level](#Similarity-Metrics-on-Attribute-Level)
    - [Table of Contents of Attribute Similarities](#Table-of-Contents-of-Attribute-Similarities)
- [DataFrame with Attributes and Similarity Features](#DataFrame-with-Attributes-and-Similarity-Features)
- [Summary](#Summary)
    - [Full Feature Matrix with Target Vector Handover](#Full-Feature-Matrix-with-Target-Vector-Handover)

## Data Takeover

Swissbib's raw data of the goldstandard has been processed in chapter [Goldstandard and Data Preparation](./2_GoldstandardDataPreparation.ipynb). As the first step of this chapter, this data is loaded for further processing to the feature matrix and target vector for the subsequent machine learning model chapters.

In [2]:
import os
import pandas as pd
import pickle as pk
import bz2
import _pickle as cPickle

path_goldstandard = './daten_goldstandard'

# Restore metadata so far
with open(os.path.join(path_goldstandard, 'columns_metadata.pkl'), 'rb') as handle:
    columns_metadata_dict = pk.load(handle)

# Restore DataFrame with features from compressed pickle file
with bz2.BZ2File((os.path.join(
    path_goldstandard, 'feature_base_df.pkl')), 'rb') as file:
    df_feature_base = cPickle.load(file)

# Extend display to number of columns of DataFrame
pd.options.display.max_columns = len(df_feature_base.columns)

df_feature_base.head()

,035liste_x,035liste_y,century_x,century_y,coordinate_E_x,coordinate_E_y,coordinate_N_x,coordinate_N_y,coordinate_x,coordinate_y,corporate_110_x,corporate_110_y,corporate_710_x,corporate_710_y,corporate_full_x,corporate_full_y,decade_x,decade_y,docid_x,docid_y,doi_x,doi_y,duplicates,edition_x,edition_y,exactDate_x,exactDate_y,format_postfix_x,format_postfix_y,format_prefix_x,format_prefix_y,isbn_x,isbn_y,ismn_x,ismn_y,masters_docid,musicid_x,musicid_y,pages_x,pages_y,part_x,part_y,person_100_x,person_100_y,person_245c_x,person_245c_y,person_700_x,person_700_y,pubinit_x,pubinit_y,pubword_x,pubword_y,pubyear_x,pubyear_y,scale_x,scale_y,ttlfull_245_x,ttlfull_245_y,ttlfull_246_x,ttlfull_246_y,ttlpart_x,ttlpart_y,volumes_x,volumes_y
0,"[(OCoLC)731635279, (ABN)000539983]","[(OCoLC)731635279, (ABN)000539983]",2009,2009,,,,,[],[],,,,,,,2009,2009,000311049,000311049,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem englischen übersetzt von...,jane austen ; aus dem englischen übersetzt von...,"grawechristian, graweursula","grawechristian, graweursula",reclam jun.,reclam jun.,[Reclam jun.],[Reclam jun.],2009,2009,,,"emma, roman","emma, roman",,,"{'245': ['Emma', 'Roman']}","{'245': ['Emma', 'Roman']}",600,600
1,"[(OCoLC)731635279, (ABN)000539983]","[(OCoLC)731635279, (NEBIS)009587153]",2009,2009,,,,,[],[],,,,,,,2009,2009,000311049,196506476,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem englischen übersetzt von...,jane austen ; aus dem engl. übers. von ursula ...,"grawechristian, graweursula",,reclam jun.,reclam,[Reclam jun.],[Reclam],2009,2009,,,"emma, roman",emma,,,"{'245': ['Emma', 'Roman']}",{'245': ['Emma']},600,600
2,"[(OCoLC)731635279, (ABN)000539983]","[(OCoLC)731635279, (LIBIB)000315536]",2009,2009,,,,,[],[],,,,,,,2009,2009,000311049,323173349,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem englischen übersetzt von...,jane austen,"grawechristian, graweursula",,reclam jun.,reclam,[Reclam jun.],[Reclam],2009,2009,,,"emma, roman","emma, roman",,,"{'245': ['Emma', 'Roman']}","{'245': ['Emma', 'Roman']}",600,600
3,"[(OCoLC)731635279, (NEBIS)009587153]","[(OCoLC)731635279, (ABN)000539983]",2009,2009,,,,,[],[],,,,,,,2009,2009,196506476,000311049,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem engl. übers. von ursula ...,jane austen ; aus dem englischen übersetzt von...,,"grawechristian, graweursula",reclam,reclam jun.,[Reclam],[Reclam jun.],2009,2009,,,emma,"emma, roman",,,{'245': ['Emma']},"{'245': ['Emma', 'Roman']}",600,600
4,"[(OCoLC)731635279, (NEBIS)009587153]","[(OCoLC)731635279, (NEBIS)009587153]",2009,2009,,,,,[],[],,,,,,,2009,2009,196506476,196506476,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem engl. übers. von ursula ...,jane austen ; aus dem engl. übers. von ursula ...,,,reclam,reclam,[Reclam],[Reclam],2009,2009,,,emma,emma,,,{'245': ['Emma']},{'245': ['Emma']},600,600


Chapter [Overview and Summary](./0_OverviewSummary.ipynb) assesses the predictions of the various models. One measure to analyse the results will be the resulting confusion matrix for a model prediction. The confusion matrix will reveal cases in the testing data that the model will predict opposite to the target values of the testing data. These predictions are called false positives and false negatives. To be able to analyse the original attribute values of the affected records in detail, columns $\texttt{035liste}\_\texttt{x}$, $\texttt{035liste}\_\texttt{y}$, $\texttt{docid}\_\texttt{x}$, and $\texttt{docid}\_\texttt{y}$ need to be restored. These four attributes will now be saved to a separate DataFrame to be reloaded in later chapters.

In [3]:
# Store docid's for fast identification of row pairs in model results
df_index_docids = df_feature_base[['035liste_x', '035liste_y', 'docid_x', 'docid_y']]
# The DataFrame of pairs with target information
df_feature_base = df_feature_base[columns_metadata_dict['columns_to_use']]

df_feature_base.sample(n=5)

,duplicates,coordinate_E_x,coordinate_E_y,coordinate_N_x,coordinate_N_y,corporate_full_x,corporate_full_y,doi_x,doi_y,edition_x,edition_y,exactDate_x,exactDate_y,format_prefix_x,format_prefix_y,format_postfix_x,format_postfix_y,isbn_x,isbn_y,ismn_x,ismn_y,musicid_x,musicid_y,part_x,part_y,person_100_x,person_100_y,person_700_x,person_700_y,person_245c_x,person_245c_y,pubinit_x,pubinit_y,scale_x,scale_y,ttlfull_245_x,ttlfull_245_y,ttlfull_246_x,ttlfull_246_y,volumes_x,volumes_y
137347,0,,,,,,,,,,,2005uuuu,2007uuuu,bk,vm,020000,010300,[0-521-82437-0],[],,,,,,,austenjane,,croninrichard,jacquetluc,jane austen ; ed. by richard cronin and doroth...,ein film von luc jacquet,cambridge univ. press,kinowelt home entertainment,,,emma,die reise der pinguine,,,600,1 82
253656,0,,,,,,uniwersytet im. adama mickiewicza w poznaniu,,,,,2017uuuu,19679999,bk,cr,010000,030600,[],[0081-6485],,,,,2017 313 74,,,,"schneiderreto u., höfercandida, haffnerpeter",,"reto u. schneider, candida höfer, peter haffne...","uniwersytet im. adama mickiewicza w poznaniu, uam",,uam,,,"alles wissen dieser welt, warum bibliotheken n...",studia historiae oeconomicae,,,74,
243340,0,,,,,schweizerische normen-vereinigung,,,,,,2016uuuu,1990uuuu,bk,mu,020053,040000,[],[],,,,426,,,,mozartwolfgang amadeus,,mozartwolfgang amadeus,,mozart,,philips classics,,,health informatics - personal health device co...,"die zauberflöte, the magic flute = la flûte en...",medizinische informatik - kommunikation von ge...,,1,1 2 1
183061,0,,,,,,,,,3,,2001uuuu,1998uuuu,bk,bk,020000,020000,[3-13-127283-X],[],,,,,,115,schusterhans-peter,mozartwolfgang amadeus,trappehans-joachim,lustigjustus,"hans-peter schuster, hans-joachim trappe",justus lustig,thieme,,,,ekg-kurs für isabel,"die zauberflöte, frei nach emanuel schikaneder...",,,323,24
107427,0,,,,,,,,,,5,2005uuuu,19631970,vm,bk,010300,020000,[],[],,,,,,,,mozartwolfgang amadeus,jacquetluc,cornelissenthilo,ein film von luc jacquet,von w. a. mozart ; dargestellt von thilo corne...,frenetic films,,,,die reise der pinguine,die zauberflöte,,,1 82,107


In [4]:
print('Number of rows labelled as duplicates {:,d}'.format(len(df_feature_base[
    df_feature_base.duplicates==1])))
print('Number of rows labelled as uniques {:,d}'.format(len(df_feature_base[
    df_feature_base.duplicates==0])))
print('Total number of rows in DataFrame {:,d}'.format(df_feature_base.shape[0],
      'number of columns', df_feature_base.shape[1]))

Number of rows labelled as duplicates 15,818
Number of rows labelled as uniques 257,955
Total number of rows in DataFrame 273,773


In [5]:
print('Part of duplicates (1) on uniques (2) in units of [%]')
print(round(100*df_feature_base.duplicates.value_counts(normalize=True), 1))

Part of duplicates (1) on uniques (2) in units of [%]
0    94.2
1     5.8
Name: duplicates, dtype: float64


DataFrame feature base is the starting point used for the further processing in this chapter.

## Object Distance and Similarity

A mathematical idea of distance and similarity is needed for understanding object pair comparison. This section starts with a motivation for calculating similarities and afterwards gives a very basic definition of the two central terms, distance and similarity. The text of this section is a summary of [[Chri2012](./A_References.ipynb#chri2012)].

### Mathematical Definitions

The attributes to be used for pair comparison may contain values of poor quality. The quality originates in the way the data has been entered at the very source. Manual data entry may suffer from mistyping, automatically scanned data may suffer from insufficiencies of the scanned base material or the recognition algorithm in the optical character recognition (OCR) processing. The basic step of a deduplication process is to identify the probability of two strings of a pair to be a pair of duplicates. This is done by calculating a similarity value between the two strings compared, rather than using an exact comparison function. Based on this common similarity value for an attribute pair, their being duplicates can be decided.

The term similarity is strongly coupled to the term of distance of two values of an attribute. Mathematically, a distance can be explained with the help of a distance function. A _distance function_ or _distance metric_ $dist(o_i, o_j)$ between two points or data objects $o_i$ and $o_j$ must fulfill four requirements.

1. $dist(o_i, o_i)=0$, the distance from an object to itself is zero.
- $dist(o_i, o_j)\ge 0$, the distance between two objects is a non-negative number.
- $dist(o_i, o_j)=dist(o_j, o_i)$, the distance between two objects is symmetric.
- $dist(o_i, o_j)\le dist(o_i, o_k)+dist(o_k, o_j)$, the triangular inequality must hold. It states that the direct distance beween two objects is never larger than the combined distance when going through a third object.

A distance value expresses the dissimilarity $d$ of two objects [[HanK2012](./A_References.ipynb#hank2012)] and can therefore be converted into a similarity value $s$, calculating $s = \frac{1}{d}$, assuming $d\gt 0$. Alternatively, assuming the distance value is normalised $0\le d\le 1$, the similarity value can be calculated to $s = 1-d$. A _similarity function_ $sim(a_i, aj)$ between two attributes which can be strings, numbers, dates, geographic locations, text, XML documents, etc. fulfills the general requirements.

1. $sim(a_i, a_i)=1$, the result of comparing a value with itself is an exact similarity.
- $sim(a_i, a_j)=0$, the similarity of values that are completely different from each other is 0. What accounts for 'complete different' depends upon the type of data that are compared.
- $0\lt sim(a_i, a_j)\lt 1$, an approximate similarity between exact similarity and total dissimilarity is calculated if two attribute values are somewhat similar to each other. What accounts for 'somewhat different' depends upon the type of data that are compared.

The dissimilarity between two objects $o_i$ and $o_j$ can be computed based on the ratio of mismatches,
$$
d(o_i, o_j) = \frac{p-m}{p},
$$
where $m$ is the number of matching attributes and $p$ is the total number of attributes describing the objects [[HanK2012](./A_References.ipynb#hank2012)]. Thus the similarity between two objects can be computed as
$$
sim(o_i, o_j) = 1 - d(o_i, o_j) = \frac{m}{p}.
$$

For data deduplication, a comparison function needs to be tailored to the type of underlying data. Although there is a correspondence between a similarity function and the mathematical concept of a distance function, not all known and implemented similarity comparison functions used for string pair comparison fulfill the requirements of a distance function. Some similarity functions are not symmetric, others do not fulfill the triangular inequality. Decision taking on the best similarity function for a string pair, will be based on the effect, a similarity function has for the purpose needed. In the case of this capstone project, this purpose is its capability to contribute to the prediction whether a pair of records is a pair duplicates or a pair of uniques.

### Library TextDistance

An internet research on string distance calculation with Python has revealed libraries [[StSi](./A_References.ipynb#stsi)], [[TeDi](./A_References.ipynb#tedi)] and seperate code snippets for distinct algorithms. After trying the referenced libraries and a downloaded code snippet for a Smith Waterman similarity [[SmWa](./A_References.ipynb#smwa)], the text distance library [[TeDi](./A_References.ipynb#tedi)] has been decided to be the best decision for this capstone project. The decision is based on the GitHub statistics of stars and the date of the latest pull requests, indicating its popularity and maintenance activity of the library. A look at the API of the library, reveals the Python library to be a complete implementation (compared to suggestions of similarity metrics in [[Chri2012](./A_References.ipynb#chri2012)]) and easy to use.

In [6]:
# Install textdistance Python library - if not done, yet.
! pip install textdistance

For using the library, see documentation in [[TeDi](./A_References.ipynb#tedi)]. For the purposes of this chapter, function $\texttt{.normalized}\_\texttt{similarity()}$ of an instantiated textdistance object will be used.

In [7]:
import textdistance as tedi

With the code line above, the library is imported for application in this chapter. In appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb) the effects of the similarity metrics of the library are compared for better understanding of their specific behaviour. This comparison for each attribute is the basis of deciding the best similarity metric available for an attribute pair.

## Similarity Metrics on Attribute Level

This section implements the decision for calculating the similarity metric for each attribute of the raw data based on appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb). The implementation is applied on a pair of attributes of two records, resulting in a new attribute, the similarity value, of the final feature matrix. A general function $\texttt{build_delta_feature}$ is provided by the code file [data_preparation_funcs.py](./data_preparation_funcs.py) for transforming two attributes into their feature attribute holding their similarity value.

In [8]:
import data_preparation_funcs as dpf

### Table of Contents of Attribute Similarities

- [coordinate](#coordinate)
- [corporate](#corporate)
- [doi](#doi)
- [edition](#edition)
- [exactDate](#exactDate)
- [format](#format)
- [isbn](#isbn)
- [ismn](#ismn)
- [musicid](#musicid)
- [part](#part)
- [person](#person)
- [pubinit](#pubinit)
- [scale](#scale)
- [ttlfull](#ttlfull)
- [volumes](#volumes)

In [9]:
# These dictionary attributes will be filled in function dpf.build_delta_feature()
columns_metadata_dict['similarity_metrics'] = {}
columns_metadata_dict['features'] = []

### coordinate

As discussed in chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{coordinate}$ holds coordinates of maps. To decide whether a map covers the same geographical range, a metric will be chosen that compares the coordinate number digits from left to right. The more digits are found to be the equal, the higher the similarity value is calculated. The comparison stops with the first digit pair that differs. This algorithm is satisfyed by the LCS (Longest Common Substring comparison) algorithm and generates the wanted result, see appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb).

In [10]:
attribute = 'coordinate'

columns_metadata_dict['similarity_metrics'][attribute+'_E'] = tedi.LCSStr()
columns_metadata_dict['similarity_metrics'][attribute+'_N'] = tedi.LCSStr()

ne_values = ['_E', '_N']

for ne in ne_values :
    df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
        df_feature_base, attribute+ne,
        columns_metadata_dict['similarity_metrics'][attribute+ne],
        columns_metadata_dict)

The length of attribute $\texttt{coordinate}$ is exactly eight digits. The distinct similarity values that may occurr form a discrete set of values with a distance of $\frac{1}{8}$ between adjacent values.

In [11]:
uniques, uniques_len = {}, {}

for ne in ne_values :
    uniques[attribute+ne], uniques_len[attribute+ne] = dpf.determine_similarity_values(
        df_feature_base, attribute+ne)

coordinate_E values range [0.    0.125 0.25  0.375 0.5   0.625 0.875 1.   ]
coordinate_N values range [0.    0.375 0.5   0.625 0.75  0.875 1.   ]


Looking at some samples of the feature matrix reveals a good match to the expectations.

In [12]:
position = 3

for ne in ne_values :
    dpf.show_samples_interval(
        df_feature_base, attribute+ne,
        uniques[attribute+ne][uniques_len[attribute+ne]-position],
        uniques[attribute+ne][uniques_len[attribute+ne]-position+1]
    )

,duplicates,coordinate_E_delta,coordinate_E_x,coordinate_E_y
55249,0,0.625,e0080855,e0080900
96184,0,0.875,e0080851,e0080855
182806,0,0.875,e0080855,e0080851
55311,0,0.875,e0080855,e0080851
865,1,0.875,e0080851,e0080855


0.625 <= coordinate_E_delta <= 0.875


,duplicates,coordinate_N_delta,coordinate_N_x,coordinate_N_y
272543,1,0.750,n0461242,n0461224
69505,0,0.750,n0460826,n0460833
865,1,0.875,n0460821,n0460826
69498,0,0.750,n0460826,n0460833
257524,0,0.750,n0460833,n0460821


0.75 <= coordinate_N_delta <= 0.875


The samples above show the wanted similarity behaviour for value ranges greater than 0. The metric has the weakness, though, that empty coordinate values, e.g. for bibliographical units other than maps, have each been calculated to a similarity of 0. Some samples for duplicates in the training data are shown below.

In [13]:
dpf.show_samples_interval(
    df_feature_base[df_feature_base.duplicates==1],
    attribute+'_E', uniques[attribute+'_E'][0], uniques[attribute+'_E'][1], 10)

,duplicates,coordinate_E_delta,coordinate_E_x,coordinate_E_y
272852,1,0.0,,
262237,1,0.0,,
261604,1,0.0,,
263093,1,0.0,,
268624,1,0.0,,
262181,1,0.0,,
273040,1,0.0,,
262520,1,0.0,,
262822,1,0.0,,
272783,1,0.0,,


0.0 <= coordinate_E_delta <= 0.125


This downside shall be avoided by marking pairs of missing coordinate values on both sides with a special negative value, which will point out to the models to be trained, the special case of missing information in a row. The implementation of this logic is done in function $\texttt{.mark}\_\texttt{missing}()$. The absolute value of this negative number is conrolled by a factor which is passed to the function as a parameter. The function handles explicitly two cases. The first one is missing information in both attributes (resulting in $-1*\texttt{factor}$) and the second one is missing information in only one of the two attributes (resulting in $-0.5*\texttt{factor}$).

In [14]:
for ne in ne_values :
    df_feature_base = dpf.mark_missing(df_feature_base, attribute+ne, factor)

### corporate

Attribute $\texttt{corporate}$ is a collection of corporate names. The Monge-Elkan metric compares string tokens pairwise [[Chri2012](./A_References.ipynb#chri2012)] while the LCS metric searches for the longest common substring. Assessing the differences of these two metrics with the help of their values distribution in chapter [Features Discussion and Dummy Classifier Baseline](./5_FeatureDiscussionDummyBaseline.ipynb), reveals a better distribution behaviour for LCS. Therefore, the LCS metric will be chosen for this attribute.

In [15]:
attribute = 'corporate_full'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.LCSStr()
#tedi.StrCmp95()
#tedi.MongeElkan()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [16]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

corporate_full values range [0.         0.01333333 0.01428571 0.01639344 0.01666667 0.01694915
 0.01818182 0.01960784 0.02       0.02040816 0.0212766  0.02173913
 0.02272727 0.02380952 0.025      0.02531646 0.02542373 0.02666667
 0.02702703 0.02727273 0.02857143 0.03       0.03030303 0.03125
 0.03225806 0.03278689 0.03333333 0.03389831 0.03448276 0.03571429
 0.03636364 0.03773585 0.03797468 0.03846154 0.03921569 0.04
 0.04081633 0.04166667 0.04237288 0.04255319 0.04285714 0.04347826
 0.04444444 0.04545455 0.046875   0.04761905 0.04878049 0.04918033
 0.05       0.05263158 0.05333333 0.05405405 0.05454545 0.05555556
 0.05660377 0.05714286 0.05882353 0.05932203 0.06       0.06060606
 0.06122449 0.0625     0.06329114 0.06363636 0.06382979 0.06451613
 0.06521739 0.06557377 0.06666667 0.06779661 0.06818182 0.06896552
 0.07       0.07142857 0.07272727 0.07317073 0.075      0.0754717
 0.07594937 0.07627119 0.07692308 0.078125   0.07843137 0.07894737
 0.08       0.08108108 0.08163265 0.08181818

Its $110$ part is sparsely filled and even its $710$ part comes along with a little more than $10\%$ of filling, only. The LCS metric generates a similarity of 1 for the cases where both strings of a pair are empty. Missing values on both sides may be an indicator for a pair of duplicates but due to the sparsely available information, it is a weak indicator. Therefore, the pairs with missing data on both sides of the pair, will be marked with the negative value.

In [17]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

Some sample cases are shown below for both $\texttt{corporate}$ features.

In [18]:
dpf.show_samples_interval(
    df_feature_base[df_feature_base.duplicates==1],
    attribute, 0.0, 1.0, 20
)

,duplicates,corporate_full_delta,corporate_full_x,corporate_full_y
264283,1,1.000000,"interkantonale lehrmittelzentrale (rapperswil,...","interkantonale lehrmittelzentrale (rapperswil,..."
272264,1,0.795918,schweizerische gesellschaft für bildungsforschung,schweizerbsche gesellschaft für bildungsforschung
264256,1,1.000000,"interkantonale lehrmittelzentrale (rapperswil,...","interkantonale lehrmittelzentrale (rapperswil,..."
1427,1,1.000000,schweizerische normen-vereinigung,schweizerische normen-vereinigung
273696,1,1.000000,allgemeine geschichtforschende gesellschaft de...,allgemeine geschichtforschende gesellschaft de...
261267,1,0.500000,eidgenössische landestopographie,eidgenössishce andestopographie
268274,1,0.536585,historischer verein des kantons solothurn,historischer verein de skantons solothurn
272253,1,0.740000,schweizerische gesellschaft für bildungsforschung,schweizerische gesellschaft für bildurngsforsc...
265424,1,1.000000,"metropolitan opera (new york)chorus, metropoli...","metropolitan opera (new york)chorus, metropoli..."
272146,1,1.000000,"interkantonale lehrmittelzentrale (luzern), st...","interkantonale lehrmittelzentrale (luzern), st..."


0.0 <= corporate_full_delta <= 1.0


In [19]:
position = uniques_len[attribute]//2 # Let's have a look in the middle range of the similarities.

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+2], 20)

,duplicates,corporate_full_delta,corporate_full_x,corporate_full_y
268504,1,0.562500,arts florissants,arts florszants
267596,1,0.562500,eidgenössische landestopographie,eidgenössische lanestopographie
272741,1,0.558824,schweizerische normen-vereinigung,schweizerischey normen-vereinigung
267590,1,0.562500,eidgenössische landestopographie,eidgenössisch landestopographie
268507,1,0.562500,arts florissants,ats flworissants
263137,1,0.558824,interkantonale lehrmittelzentrale,interkantconale lehrmittelzenrtale
263986,1,0.562500,arts florissants,arts florssants
265419,1,0.560000,"metropolitan opera (new york)chorus, metropoli...","metropolitan opera (new york)chorus, metroodli..."
272722,1,0.558824,schweizerische normen-vereinigung,schweizmgrische normen-vereiingung
267711,1,0.562500,eidgenössische landestopographie,eidgenössischl landestopographie


0.5588235294117647 <= corporate_full_delta <= 0.5625


### doi

Swissbib uses an explicit $\texttt{doi}$ attribute for its deduplication implementation. In chapter [Goldstandard and Data Preparation](./2_GoldstandardDataPreparation.ipynb), the real doi identifier has been isolated with the help of a preprocessing function $\texttt{.reduce}\_\texttt{to}\_\texttt{doi}\_\texttt{element()}$, see [Data Analysis](./1_DataAnalysis.ipynb). Attribute $\texttt{doi}$ contains a single string value. The Identity metric will be used for comparing the string values of a pair in a row, calculating a similarity value of 1.0 or 0.0 for each pair. If one list is empty a value of 0 is returned.

In [20]:
attribute = 'doi'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Identity()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

df_feature_base['doi_delta'].unique()

array([1., 0.])

Some sample cases are shown below for each category of $\texttt{doi}\_\texttt{delta}$.

In [21]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

for doi_delta_value in df_feature_base['doi_delta'].unique():
    number_of_max_samples = min(
        10,
        len(df_feature_base[df_feature_base['doi_delta']==doi_delta_value])
    )

    dpf.show_samples_distinct(df_feature_base, 'doi', doi_delta_value, number_of_max_samples)
    print(f'doi_delta = {doi_delta_value}')

doi values range [0. 1.]


,duplicates,doi_delta,doi_x,doi_y
227751,0,1.0,,
259509,1,1.0,,
211449,0,1.0,,
51468,0,1.0,,
251625,0,1.0,,
238883,0,1.0,,
158040,0,1.0,,
164946,0,1.0,,
191473,0,1.0,,
36081,0,1.0,,


doi_delta = 1.0


,duplicates,doi_delta,doi_x,doi_y
13737,0,0.0,,10.5169/seals-515343
163173,0,0.0,,10.5169/seals-515343
211178,0,0.0,,10.5169/seals-377251
20237,0,0.0,,10.5169/seals-377392
167674,0,0.0,10.1093/cid/ciu795,
39185,0,0.0,,10.5169/seals-376396
160752,0,0.0,,10.5169/seals-376473
193016,0,0.0,10.5451/unibas-006499413,
154846,0,0.0,10.5167/uzh-53042,10.5169/seals-377332
70639,0,0.0,,10.5169/seals-377305


doi_delta = 0.0


In [22]:
# Let's have a look at some non-empty doi elements
df_doi_with_element = df_feature_base[df_feature_base.doi_x.apply(lambda x : len(x) > 0)]

for doi_delta_value in df_feature_base['doi_delta'].unique():
    number_of_max_samples = min(
        10,
        len(df_feature_base[df_feature_base['doi_delta']==doi_delta_value])
    )

    dpf.show_samples_distinct(df_doi_with_element, 'doi', doi_delta_value, number_of_max_samples)
    print(f'doi_delta = {doi_delta_value}')

,duplicates,doi_delta,doi_x,doi_y
267108,1,1.0,10.5169/seals-515343,10.5169/seals-515343
265769,1,1.0,10.5167/uzh-67756,10.5167/uzh-67756
266650,1,1.0,10.5169/seals-377362,10.5169/seals-377362
272537,1,1.0,10.5451/unibas-006499413,10.5451/unibas-006499413
266480,1,1.0,10.5169/seals-377079,10.5169/seals-377079
266282,1,1.0,10.5169/seals-376645,10.5169/seals-376645
262860,1,1.0,10.3931/e-rara-61897,10.3931/e-rara-61897
266329,1,1.0,10.5169/seals-376773,10.5169/seals-376773
266487,1,1.0,10.5169/seals-377079,10.5169/seals-377079
262189,1,1.0,10.1093/cid/cir669,10.1093/cid/cir669


doi_delta = 1.0


,duplicates,doi_delta,doi_x,doi_y
193224,0,0.0,10.5451/unibas-006499413,
194444,0,0.0,10.5451/unibas-006503313,
207823,0,0.0,10.1093/cid/ciu795,
200168,0,0.0,10.1007/978-3-642-41698-9,
207667,0,0.0,10.1093/cid/ciu795,
208913,0,0.0,10.1093/ndt/gft319,
208020,0,0.0,10.1093/cid/cir669,
165872,0,0.0,10.1093/cid/cir669,
255450,0,0.0,10.1055/b-005-143650,
167044,0,0.0,10.1007/978-3-642-41698-9,


doi_delta = 0.0


As can be seen above, a value of 1.0 is returned if both strings of a pair are empty. As the attribute filling of $\texttt{doi}$ is sparse, see chapter [Data Analysis](./1_DataAnalysis.ipynb), the $\texttt{doi}\_\texttt{delta}$ indicates strongly a pair of duplicates for most rows. To avoid such misleading identity indication, function $\texttt{.mark}\_\texttt{missing()}$ will be applyed to the attribute.

In [23]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

### edition

In its original form in Swissbib's raw data, the edition statement is a string value which may have several words. The modelling on this attribute has been tried with and without stripping letter characters from the string. The final decision for the best processing will be documented in chapter [Overview and Summary](./0_OverviewSummary.ipynb). A Jaccard similarity is tried for this attribute.

In [24]:
attribute = 'edition'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Jaccard()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [25]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

import numpy as np

edition_delta_uniques = np.sort(df_feature_base['edition_delta'].unique())
edition_delta_uniques_len = len(edition_delta_uniques)
print('edition values range', edition_delta_uniques[:30])

edition values range [0.         0.125      0.14285714 0.16666667 0.2        0.25
 0.28571429 0.33333333 0.4        0.5        0.6        1.        ]
edition values range [0.         0.125      0.14285714 0.16666667 0.2        0.25
 0.28571429 0.33333333 0.4        0.5        0.6        1.        ]


The comparison results in a wide number of distinct similarity values for the goldstandard data set. Below, some examples are shown.

In [26]:
position = edition_delta_uniques_len

dpf.show_samples_interval(
    df_feature_base, 'edition',
    edition_delta_uniques[edition_delta_uniques_len-position-2],
    edition_delta_uniques[edition_delta_uniques_len-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, 'edition',
    edition_delta_uniques[edition_delta_uniques_len-position],
    edition_delta_uniques[edition_delta_uniques_len-position+2], 10)

position = edition_delta_uniques_len//2

dpf.show_samples_interval(
    df_feature_base, 'edition',
    edition_delta_uniques[edition_delta_uniques_len-position-2],
    edition_delta_uniques[edition_delta_uniques_len-position-1], 10)

,duplicates,edition_delta,edition_x,edition_y
222949,0,1.0,,
143776,0,1.0,,
186797,0,1.0,,
10101,0,1.0,,
34731,0,1.0,,
37334,0,1.0,,
5415,0,1.0,,
179578,0,1.0,,
252310,0,1.0,,
110632,0,1.0,,


0.6 <= edition_delta <= 1.0


,duplicates,edition_delta,edition_x,edition_y
102908,0,0.0,10,
5681,0,0.0,,2
29052,0,0.0,5,
184601,0,0.0,1863,
188963,0,0.0,3,
55195,0,0.0,1,
70535,0,0.0,,2
5913,0,0.0,,3
140303,0,0.0,,2
67946,0,0.0,,2


0.0 <= edition_delta <= 0.1428571428571428


,duplicates,edition_delta,edition_x,edition_y
4531,0,0.25,1,1899
105089,0,0.25,8,1928
185716,0,0.25,8,1889
54939,0,0.25,1,1907
71540,0,0.25,8,1899
103333,0,0.20,10,1899
97237,0,0.25,1791,1
248220,0,0.20,10425,2
189283,0,0.25,3,1943
4757,0,0.25,1,1994


0.19999999999999996 <= edition_delta <= 0.25


Again, for $\texttt{edition}\_\texttt{delta} = 1$, many empty values of the $\texttt{edition}$ attribute can be observed. These will be marked with the special negative value in the data with the goal to distinguish them from the matching attribute pairs.

In [27]:
df_feature_base = dpf.mark_missing(df_feature_base, 'edition', factor)

In [28]:
position = edition_delta_uniques_len

dpf.show_samples_interval(
    df_feature_base, 'edition',
    edition_delta_uniques[edition_delta_uniques_len-position-2],
    edition_delta_uniques[edition_delta_uniques_len-position-1], 10)

,duplicates,edition_delta,edition_x,edition_y
267445,1,1.0,1907,1907
263155,1,1.0,1994,1994
183199,0,1.0,3,3
263100,1,1.0,3,3
264674,1,1.0,4,4
269669,1,1.0,7,7
270713,1,1.0,3,3
264543,1,1.0,1,1
260665,1,1.0,4,4
259742,1,1.0,3,3


0.6 <= edition_delta <= 1.0


### exactDate

As discussed in chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{exactDate}$ holds a year number stored in the first four digits. Letter 'u' is used as a placeholder for an unknown digit. The attribute may hold some month and day or a second year information in its second four digits, additionally.

The attribute will be kept as a string and will not be transformed to an integer. The feature attribute of the record pair to be compared will be calculated with a modified Hamming algorithm, see appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb). The resulting similarity will be stored in a new attribute $\texttt{exactDate}\_\texttt{delta}$ which will be taken for the model calculation.

As can be seen in chapter [Decision Tree Model](./6_DecisionTreeModel.ipynb), this attribute is important for prediction. Different ways of increasing the weight of the unknown status of a digit have been tried. The different ways can be seen in the implementations below. The algorithm decided for the final simulation will be documented in chapter [Overview and Summary](./0_OverviewSummary.ipynb).

In [29]:
import string

def no_xor (x_side, y_side) :
    number = 0
    for i in range(len(x_side)) :
        if ((x_side[i] in string.ascii_lowercase) | (y_side[i] in string.ascii_lowercase)) & (x_side[i] != y_side[i]) :
            number = number + 1
    return number

print('Example comparison results in a value of', no_xor ('202a0aaa', '1920uuuu'))

Example comparison results in a value of 5


In [30]:
attribute = 'exactDate'

# Replace letter 'u' with letter 'a' for one of the two strings.
#  As an effect, the resulting Hamming similarity for a letter
#  instead of a numerical digit in either string will add with an amount 0.
df_feature_base[attribute+'_x'] = df_feature_base.exactDate_x.str.replace('u', 'a')

# Compute Hamming similarity for century string pair.
columns_metadata_dict['similarity_metrics'][attribute] = tedi.Hamming()

unknown_share = 16

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

if exactDate_mode == 'added_u':
    # Add amount of 1/16 to Hamming similarity for every letter digit.
    #  But only maximum number of letter digits in both strings of a pair.
    df_feature_base[attribute+'_delta'] = df_feature_base[[
        attribute+'_x', attribute+'_y', attribute+'_delta']].apply(
        lambda x : x[attribute+'_delta'] + 
        max(x[attribute+'_x'].count('a'), x[attribute+'_y'].count('u'))/unknown_share, axis=1
    )
elif exactDate_mode == 'xor':
    # Add amount of 1/16 to Hamming similarity for every letter digit.
    #  But only number of position-wise xor-ed letter digits in the two strings of a pair.
    df_feature_base[attribute+'_delta'] = df_feature_base[[
        attribute+'_x', attribute+'_y', attribute+'_delta']].apply(
        lambda x : x[attribute+'_delta'] + 
        no_xor(x[attribute+'_x'], x[attribute+'_y'])/unknown_share, axis=1
    )

In [31]:
df_feature_base[['exactDate_x', 'exactDate_y', 'exactDate_delta']].sample(n=10)

,exactDate_x,exactDate_y,exactDate_delta
188240,2013aaaa,2007uuuu,0.500
169117,2012aaaa,2000uuuu,0.500
78613,2005aaaa,uuuuuuuu,0.500
180400,20091990,1932uuuu,0.250
36590,aaaaaaaa,1998uuuu,0.500
121528,19911794,1997uuuu,0.625
267267,1891aaaa,1891uuuu,0.750
193488,2016aaaa,1901uuuu,0.250
99038,19702006,1960uuuu,0.625
242028,1959aaaa,2015uuuu,0.250


All resulting values of equal strings are equal to 1.

In [32]:
df_feature_base[['exactDate_x', 'exactDate_y', 'exactDate_delta']][
    df_feature_base.exactDate_x == df_feature_base.exactDate_y
].sort_values('exactDate_delta', ascending=False).head()

,exactDate_x,exactDate_y,exactDate_delta
159,20022000,20022000,1.0
268889,20151475,20151475,1.0
268886,20151475,20151475,1.0
268884,20151475,20151475,1.0
268883,20151475,20151475,1.0


A discrete set of different similarity values can be found in the attribute deltas. Some sample records are shown below.

In [33]:
exactDate_deltas = np.sort(df_feature_base.exactDate_delta.unique())
exactDate_deltas

array([0.    , 0.125 , 0.25  , 0.3125, 0.375 , 0.4375, 0.5   , 0.5625,
       0.625 , 0.6875, 0.75  , 0.875 , 1.    ])

In [34]:
sample_size = 5

for i in exactDate_deltas :
    dpf.show_samples_distinct(df_feature_base, 'exactDate', i, sample_size)
    print(f'exactDate_delta = {i}')

,duplicates,exactDate_delta,exactDate_x,exactDate_y
186743,0,0.0,20151475,19989999
154351,0,0.0,18501875,20012004
79620,0,0.0,20062005,19289999
30570,0,0.0,20071990,19942008
38180,0,0.0,19791999,20130627


exactDate_delta = 0.0


,duplicates,exactDate_delta,exactDate_x,exactDate_y
17047,0,0.125,19911990,20022010
207282,0,0.125,20150201,19959999
175354,0,0.125,20159999,18631869
207868,0,0.125,20111201,19819999
53212,0,0.125,19702006,18971989


exactDate_delta = 0.125


,duplicates,exactDate_delta,exactDate_x,exactDate_y
19765,0,0.25,1970aaaa,2014uuuu
160042,0,0.25,2014aaaa,19961997
19839,0,0.25,1970aaaa,2006uuuu
157011,0,0.25,18761920,2001uuuu
2516,0,0.25,2000aaaa,1965uuuu


...

exactDate_delta = 0.75


,duplicates,exactDate_delta,exactDate_x,exactDate_y
260221,1,0.875,19aa9999,19uu9999
260209,1,0.875,19aa9999,19uu9999
123788,0,0.875,19829999,19819999
271139,1,0.875,19001996,19061996
129185,0,0.875,18801890,18871890


exactDate_delta = 0.875


,duplicates,exactDate_delta,exactDate_x,exactDate_y
259468,1,1.0,17931797,17931797
272965,1,1.0,19739999,19739999
268983,1,1.0,19669999,19669999
262448,1,1.0,19679999,19679999
1303,1,1.0,20159999,20159999


exactDate_delta = 1.0


### format

Due to the discussion in chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{format}$ has been split up into two new attributes $\texttt{format}\_\texttt{prefix}$ and $\texttt{format}\_\texttt{postfix}$ which will be compared by a different similarity metrics.

- As the quality of $\texttt{format}\_\texttt{prefix}$ is expected to be high, an identity comparison should be sufficient.
- Due to the observed structure of $\texttt{format}\_\texttt{postfix}$, a q-gram based comparison will be chosen.

In [35]:
attribute = 'format'

columns_metadata_dict['similarity_metrics'][attribute+'_prefix'] = tedi.Identity()
columns_metadata_dict['similarity_metrics'][attribute+'_postfix'] = tedi.Jaccard(qval=2)

pfix_values = ['_prefix', '_postfix']

for pf in pfix_values :
    df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
        df_feature_base, attribute+pf,
        columns_metadata_dict['similarity_metrics'][attribute+pf],
        columns_metadata_dict)

In [36]:
for i in df_feature_base.format_prefix_delta[
    df_feature_base.format_prefix_x != df_feature_base.format_prefix_y].unique():
    
    dpf.show_samples_distinct(df_feature_base, 'format_prefix', i)
    print(f'format_prefix_delta = {i}')

,duplicates,format_prefix_delta,format_prefix_x,format_prefix_y
136970,0,0.0,mu,bk
200252,0,0.0,bk,mu
27063,0,0.0,bk,mp
97783,0,0.0,vm,mu
152406,0,0.0,mu,bk


format_prefix_delta = 0.0


In [37]:
for i in df_feature_base.format_postfix_delta[
    df_feature_base.format_postfix_x != df_feature_base.format_postfix_y].unique():
    
    dpf.show_samples_distinct(df_feature_base, 'format_postfix', i)
    print(f'format_postfix_delta = {i}')

,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
127827,0,0.428571,010200,020000
88482,0,0.428571,040100,010300
246162,0,0.428571,010000,010053
223247,0,0.428571,020000,020053
252327,0,0.428571,010000,040100


format_postfix_delta = 0.4285714285714286


,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
200349,0,0.111111,020053,030000
164418,0,0.111111,010100,020053
87355,0,0.111111,030600,020000
244443,0,0.111111,020053,010300
36422,0,0.111111,040100,020053


format_postfix_delta = 0.11111111111111116


,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
38903,0,0.25,020000,020353
106900,0,0.25,020300,030653
132597,0,0.25,020000,020353
145051,0,0.25,020000,020353
20464,0,0.25,010347,040100


...

format_postfix_delta = 0.0


,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
157512,0,1.0,020000,020000
130351,0,1.0,020000,020000
271127,1,1.0,010000,010000
84557,0,1.0,020000,020000
56062,0,1.0,010300,030100


format_postfix_delta = 1.0


,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
259709,1,0.666667,040100,400100
265234,1,0.666667,020000,200000
269553,1,0.666667,020000,200000
273409,1,0.666667,020000,200000
272547,1,0.666667,010700,100700


format_postfix_delta = 0.6666666666666666


### isbn

Swissbib uses each string element of the $\texttt{isbn}$ list separately for comparing with each string element of its comparison $\texttt{isbn}$ list. If two bibliographic units hold at least one element in common, this is interpreted as a strong indicator for duplicates [[WiCo2001](./A_References.ipynb#wico2001)].

This hard logic is used in a modified way in the context of this capstone project. A special comparison function $\texttt{.build}\_\texttt{delta}\_\texttt{isbn()}$ has been implemented that compares each list element of the left-hand side with each list element of the right-hand side of a pair. According to Swissbib's implementation, the Identity metric is used for string comparison, calculating a similarity value of 1.0 or 0.0 for each list element pair. For normalisation reasons, the sum of similarity values is divided by the number of elements of the smaller list. If both lists are empty a value of 1.0 is returned. If only one list is empty a value of 0.0 is returned.

In [38]:
attribute = 'isbn'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Identity()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

df_feature_base[attribute+'_delta'].unique()

array([1. , 0. , 0.5])

Some sample cases are shown below for each category of $\texttt{isbn_delta}$.

In [39]:
for isbn_delta_value in df_feature_base['isbn_delta'].unique():
    number_of_max_samples = min(
        10,
        len(df_feature_base[df_feature_base['isbn_delta']==isbn_delta_value])
    )

    dpf.show_samples_distinct(df_feature_base, 'isbn', isbn_delta_value, number_of_max_samples)
    print(f'isbn_delta = {isbn_delta_value}')

,duplicates,isbn_delta,isbn_x,isbn_y
86118,0,1.0,[],[]
79529,0,1.0,[],[]
75050,0,1.0,[],[]
110468,0,1.0,[],[]
203598,0,1.0,[],[]
121175,0,1.0,[],[]
267063,1,1.0,[],[]
205891,0,1.0,[],[]
212834,0,1.0,[],[]
262609,1,1.0,[],[]


isbn_delta = 1.0


,duplicates,isbn_delta,isbn_x,isbn_y
26289,0,0.0,[0252-9955],[978-2-7599-0190-6]
178922,0,0.0,"[978-3-7815-1531-4, 3-7815-1531-1]",[]
253629,0,0.0,[],[0081-6485]
124796,0,0.0,[],"[978-3-598-31516-9 (print), 978-3-11-097838-4]"
200756,0,0.0,"[88-7922-121-3, 978-88-7922-121-4]",[978-3-7815-1513-0]
108528,0,0.0,[3-13-127283-X],[]
103647,0,0.0,[],"[3-906721-44-2 (Livre de l'élève, BLMV), 3-906..."
192963,0,0.0,[],"[978-3-598-31486-5 (print), 978-3-11-097416-4]"
136988,0,0.0,[],[978-3-906721-38-5]
119223,0,0.0,[],[978-3-648-04700-2]


isbn_delta = 0.0


,duplicates,isbn_delta,isbn_x,isbn_y
1195,1,0.5,"[978-3-13-127286-7, 3-13-127286-4]","[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]"
1210,1,0.5,"[978-3-13-127286-7, 3-13-127286-4]","[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]"
1205,1,0.5,"[978-3-13-127286-7, 3-13-127286-4]","[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]"
1202,1,0.5,"[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]","[978-3-13-127286-7, 3-13-127286-4]"
1199,1,0.5,"[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]","[978-3-13-127286-7, 3-13-127286-4]"
1201,1,0.5,"[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]","[978-3-13-127286-7, 3-13-127286-4]"


isbn_delta = 0.5


For attribute $\texttt{isbn}$, the special marking of missing values is omitted.

### ismn

This attribute will be processed with the identity similarity metric. The reasoning for this decision is the same as for similar attributes above. 

In [40]:
attribute = 'ismn'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Identity()
#tedi.Jaccard()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [41]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

ismn values range [0. 1.]


In [42]:
for ismn_delta_value in df_feature_base[attribute+'_delta'].unique():
    number_of_max_samples = min(
        10,
        len(df_feature_base[df_feature_base[attribute+'_delta']==ismn_delta_value])
    )

    dpf.show_samples_distinct(df_feature_base, 'ismn', ismn_delta_value, number_of_max_samples)
    print(f'ismn_delta = {ismn_delta_value}')

,duplicates,ismn_delta,ismn_x,ismn_y
153045,0,1.0,,
30948,0,1.0,,
62028,0,1.0,,
54390,0,1.0,,
52360,0,1.0,,
37458,0,1.0,,
150703,0,1.0,,
13161,0,1.0,,
21403,0,1.0,,
191980,0,1.0,,


ismn_delta = 1.0


,duplicates,ismn_delta,ismn_x,ismn_y
114844,0,0.0,"m006546756 (kritischer bericht, leinen)",
256183,0,0.0,"m006546756 (kritischer bericht, leinen)",
87147,0,0.0,,m006450510
115214,0,0.0,"m006546756 (kritischer bericht, leinen)",
119227,0,0.0,,m008060205
228339,0,0.0,,m008060205
135334,0,0.0,m006450510,
126938,0,0.0,,m006450510
229336,0,0.0,,9790200207781
77528,0,0.0,,9790200207781


ismn_delta = 0.0


As can be seen in the previous chapters, attribute $\texttt{ismn}$ is filled sparsely. A lot of missing values calculate to a value of 1.0 in the chosen similarity metrics. To mark these cases specifically, they will be transformed to a negative value.

In [43]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

### musicid

Chapter [Data Analysis](./1_DataAnalysis.ipynb) shows that attribute $\texttt{musicid}$ is an identifyer for a music record. A Jaccard metric has been tested on this attribute, resulting in a distribution of many high similarity values on uniques. Comparing this result with the LCS metric, the latter has been decided.

In [44]:
attribute = 'musicid'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.LCSStr()
#tedi.Jaccard()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [45]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

musicid values range [0.         0.125      0.14285714 0.16666667 0.2        0.25
 0.28571429 0.33333333 0.375      0.4        0.42857143 0.5
 0.57142857 0.6        0.625      0.66666667 0.71428571 0.75
 0.8        0.83333333 0.85714286 1.        ]


In [46]:
position = uniques_len[attribute]

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position-2],
    uniques[attribute][uniques_len[attribute]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+2], 10)

position = uniques_len[attribute]//2

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+1], 10)

,duplicates,musicid_delta,musicid_x,musicid_y
249558,0,1.0,4553,4553
259988,1,1.0,4553,4553
259798,1,1.0,435,435
520,1,1.0,4553,4553
1045,1,1.0,4553,4553
271342,1,1.0,245,245
265562,1,1.0,5714,5714
265711,1,1.0,502430,502430
271422,1,1.0,3714,3714
260488,1,1.0,8,8


0.8571428571428572 <= musicid_delta <= 1.0


,duplicates,musicid_delta,musicid_x,musicid_y
79154,0,0.0,,
121362,0,0.0,,
113994,0,0.0,502023,
1667,0,0.0,,
77633,0,0.0,,
164392,0,0.0,4553,
173846,0,0.0,,
271739,1,0.0,,
29816,0,0.0,,
183208,0,0.0,,


0.0 <= musicid_delta <= 0.1428571428571429


,duplicates,musicid_delta,musicid_x,musicid_y
261567,1,0.5,4355,455
76682,0,0.5,4553,4355
153179,0,0.5,4553,4355
98912,0,0.5,4553,4355
256365,0,0.5,4553,155
7551,0,0.5,501326,8501
53203,0,0.5,4553,4355
114923,0,0.5,4553,4355
75144,0,0.5,4553,4355
79283,0,0.5,501326,8501


0.5 <= musicid_delta <= 0.5714285714285714


In [47]:
dpf.show_samples_interval(
    df_feature_base[df_feature_base.duplicates==1], attribute,
    uniques[attribute][0],
    uniques[attribute][uniques_len[attribute]-1], 20)

,duplicates,musicid_delta,musicid_x,musicid_y
261557,1,1.00,4355,4355
270778,1,0.00,,
260441,1,0.00,,
267233,1,0.00,,
665,1,0.00,,
263779,1,0.00,,
266563,1,0.00,,
264936,1,0.00,,
260905,1,0.00,,
265365,1,0.00,,


0.0 <= musicid_delta <= 1.0


The attribute is filled with a degree of below $10\%$. The chosen metric for it results in a similarity value of 1.0 for empty value pairs. This effect can be adjusted with function $\texttt{.mark}\_\texttt{missing}()$ as above. 

In [48]:
df_feature_base = dpf.mark_missing(df_feature_base, 'musicid', factor)

### part

Analogous to attribute $\texttt{edition}$ described above, the string value of this attribute can be stripped to pure number digits. Both ways, with and without letter stripping have been tried for modelling. The final decision for the best processing will be documented in chapter [Overview and Summary](./0_OverviewSummary.ipynb). Three different metrics have been tried for attribute $\texttt{part}$. Finally, metric Jaro will be used.

In [49]:
attribute = 'part'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.StrCmp95()
#tedi.Jaro()
#tedi.Hamming()
#tedi.LCSStr()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [50]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

part values range [0.         0.27407407 0.28240741 0.29202279 0.30740741 0.31318681
 0.31481481 0.32407407 0.32898551 0.33333333 0.33597884 0.33838384
 0.33921569 0.34444444 0.3452381  0.35128205 0.35185185 0.35897436
 0.36111111 0.36231884 0.36666667 0.37037037 0.37179487 0.37254902
 0.37301587 0.37407407 0.375      0.37777778 0.38333333 0.38461538
 0.38568376 0.38888889 0.38927739 0.39070048 0.39215686 0.39393939
 0.3952381  0.3960114  0.39646465 0.39722222 0.39814815 0.4
 0.4031746  0.4037037  0.40604575 0.40740741 0.40842491 0.40855763
 0.41025641 0.41111111 0.41125541 0.41176471 0.41203704 0.41282051
 0.41388889 0.41449275 0.41452991 0.41507937 0.41666667 0.4178744
 0.41798942 0.41851852 0.41919192 0.42083333 0.42222222 0.42261905
 0.42390289 0.4241453  0.42564103 0.42592593 0.42777778 0.42810458
 0.42857143 0.42948718 0.43030303 0.43055556 0.43115942 0.43174603
 0.43236715 0.43333333 0.43518519 0.43557423 0.43589744 0.43627451
 0.43650794 0.43703704 0.4375     0.43813131 0.43888

In [51]:
position = uniques_len[attribute]

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position-2],
    uniques[attribute][uniques_len[attribute]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+2], 10)

position = uniques_len[attribute]//7

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position-2],
    uniques[attribute][uniques_len[attribute]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+2], 10)

,duplicates,part_delta,part_x,part_y
273306,1,1.0,63,63
258976,0,1.0,,
38242,0,1.0,,
267762,1,1.0,23 23 1909 23,23 23 1909 23
135791,0,1.0,,
271916,1,1.0,,
240252,0,1.0,,
96522,0,1.0,,
260519,1,1.0,,
263739,1,1.0,4,4


0.9215686274509803 <= part_delta <= 1.0


,duplicates,part_delta,part_x,part_y
233896,0,0.0,,19 19
241849,0,0.0,2620,
144746,0,0.0,,267 1988
96248,0,0.0,285 1963,
213555,0,0.0,,288 2009
134304,0,0.0,,552 552
57791,0,0.0,,39 39
15241,0,0.0,2620,
23482,0,0.0,,90041
221729,0,0.0,,7


0.0 <= part_delta <= 0.28240740740740744


,duplicates,part_delta,part_x,part_y
216177,0,0.714286,552 552,2
227622,0,0.714286,285 285,2
257203,0,0.714286,23 1862,2
197367,0,0.714286,23 1862,2
253567,0,0.714646,2017 313 74,2002 107 117
227757,0,0.714286,285 285,5
238504,0,0.714286,2,285 285
257162,0,0.714286,23 1862,2
123315,0,0.714286,5,552 552
117917,0,0.714286,552 552,2


0.7142857142857143 <= part_delta <= 0.7146464646464646


,duplicates,part_delta,part_x,part_y
197596,0,0.717338,23 1862,23 23 1870 23
37481,0,0.716667,5 19,15 15 1926
197585,0,0.717338,23 1862,23 23 1946 23
166866,0,0.716667,28 10 2013 2421 2431,2 2
197570,0,0.717338,23 1862,23 23 1906 23
256298,0,0.716667,5 19,15 15 1926
257493,0,0.717338,23 1862,23 23 1899 23
183898,0,0.716667,26 2000 26 2000,281 2002
183903,0,0.716667,26 2000 26 2000,286 2007
115164,0,0.716667,5 19,15 15 1926


0.7159544159544161 <= part_delta <= 0.7173382173382173


In this attribute, too, moving pairs of empty values to negative values will result in a clearer distinction between pairs of uniques and duplicates, as will be seen in the graphical comparison of capter [Features Discussion and Dummy Classifier Baseline](./5_FeatureDiscussionDummyBaseline.ipynb).

In [52]:
df_feature_base = dpf.mark_missing(df_feature_base, 'part', factor)

### person

As a result of chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{person}$ has been split into three specific attributes. Attribute $\texttt{person}\_{100}$ and $\texttt{person}\_{700}$ hold strongly standardised string values. For comparing pure strings, a Levenshtein metric is recommended [[Chri2012](./A_References.ipynb#chri2012)]. Unfortunately, this metric shows a very long calculation time on the data of the capstone project. Comparing the similarity values of the Levenshtein metric with the similarity values of other metrics in appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb), similarity metric StrCmp95 has been decided to use.

In [53]:
attribute = 'person'

columns_metadata_dict['similarity_metrics'][attribute+'_100'] = tedi.StrCmp95()
columns_metadata_dict['similarity_metrics'][attribute+'_700'] = tedi.StrCmp95()
#tedi.Levenshtein()

pe_values = ['_100', '_700']

for pe in pe_values :
    print('Calculating person'+pe)
    df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
        df_feature_base, attribute+pe,
        columns_metadata_dict['similarity_metrics'][attribute+pe],
        columns_metadata_dict)

Calculating person_100
Calculating person_700


In [54]:
pe = '_100'

uniques[attribute+pe], uniques_len[attribute+pe] = dpf.determine_similarity_values(
    df_feature_base, attribute+pe)

person_100 values range [0.         0.31944444 0.32777778 ... 0.99090909 0.99130435 1.        ]


In [55]:
position = uniques_len[attribute+pe]

dpf.show_samples_interval(
    df_feature_base, attribute+pe,
    uniques[attribute+pe][uniques_len[attribute+pe]-position-2],
    uniques[attribute+pe][uniques_len[attribute+pe]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute+pe,
    uniques[attribute+pe][uniques_len[attribute+pe]-position],
    uniques[attribute+pe][uniques_len[attribute+pe]-position+2], 10)

,duplicates,person_100_delta,person_100_x,person_100_y
262595,1,1.000000,,
263584,1,1.000000,,
141557,0,1.000000,,
21052,0,1.000000,mozartwolfgang amadeus,mozartwolfgang amadeus
50784,0,1.000000,,
124662,0,1.000000,,
259881,1,0.991304,mozartwolfgang amadeus,mozartwolfgang amadeugs
142609,0,1.000000,mozartwolfgang amadeus,mozartwolfgang amadeus
221328,0,1.000000,,
55295,0,1.000000,,


0.9913043478260869 <= person_100_delta <= 1.0


,duplicates,person_100_delta,person_100_x,person_100_y
33033,0,0.0,fluryandreas,
124563,0,0.0,,mozartwolfgang amadeus
75265,0,0.0,mozartwolfgang amadeus,
56521,0,0.0,schlöndorffvolker,
71656,0,0.0,,schusterhans-peter
185112,0,0.0,,bührerwalter
237743,0,0.0,fluryandreas,
6765,0,0.0,,mortzfeldpeter
134551,0,0.0,mozartwolfgang amadeus,
194012,0,0.0,fluryandreas,


0.0 <= person_100_delta <= 0.3277777777777777


For comparing person names, like in attribute $\texttt{person}\_{245c}$, a Jaro metric will be tested [[Chri2012](./A_References.ipynb#chri2012)].

In [56]:
pe = '_245c'

columns_metadata_dict['similarity_metrics'][attribute+pe] = tedi.Jaro()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute+pe,
    columns_metadata_dict['similarity_metrics'][attribute+pe],
    columns_metadata_dict)

In [57]:
uniques[attribute+pe], uniques_len[attribute+pe] = dpf.determine_similarity_values(
    df_feature_base, attribute+pe)

person_245c values range [0.         0.24632035 0.25462963 ... 0.99881376 0.99881797 1.        ]


In [58]:
position = uniques_len[attribute+pe]

dpf.show_samples_interval(
    df_feature_base, attribute+pe,
    uniques[attribute+pe][uniques_len[attribute+pe]-position-2],
    uniques[attribute+pe][uniques_len[attribute+pe]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute+pe,
    uniques[attribute+pe][uniques_len[attribute+pe]-position],
    uniques[attribute+pe][uniques_len[attribute+pe]-position+2], 10)

,duplicates,person_245c_delta,person_245c_x,person_245c_y
64858,0,1.0,,
259737,1,1.0,jane austen ; aus dem engl. von horst höckendorf,jane austen ; aus dem engl. von horst höckendorf
150197,0,1.0,mozart,mozart
263818,1,1.0,sigrid kessler... [et al.] ; [éd.:] interkanto...,sigrid kessler... [et al.] ; [éd.:] interkanto...
201441,0,1.0,,
268908,1,1.0,w. a. mozart ; herausgegeben von/edited by ger...,w. a. mozart ; herausgegeben von/edited by ger...
270265,1,1.0,"mortzfeld, peter; raabe, paul","mortzfeld, peter; raabe, paul"
270257,1,1.0,"mortzfeld, peter; raabe, paul","mortzfeld, peter; raabe, paul"
268019,1,1.0,erland erdmann (hrsg.),erland erdmann (hrsg.)
268634,1,1.0,livret d'emanuel schikaneder ; illustré de 36 ...,livret d'emanuel schikaneder ; illustré de 36 ...


0.9988179669030733 <= person_245c_delta <= 1.0


,duplicates,person_245c_delta,person_245c_x,person_245c_y
182712,0,0.0,,[walter bührer]
136188,0,0.0,[hrsg.:] interkantonale lehrmittelzentrale luz...,
55114,0,0.0,,sigrid kessler... [et al.] ; [hrsg.:] interkan...
96112,0,0.0,,[walter bührer]
74048,0,0.0,"w. a. mozart ; klavierauszug, nach dem in der ...",
65622,0,0.0,[voltaire] ; critical edition by john renwick,
89431,0,0.0,,"sigrid keller, caty laubscher, helen wallimann..."
243987,0,0.0,,"mortzfeld, peter; raabe, paul"
210310,0,0.0,,von mozart ; dichtung von emanuel schikaneder
131569,0,0.0,sigrid kessler... [et al.] ; [éd.:] interkanto...,


0.0 <= person_245c_delta <= 0.25462962962962954


The similarities of all three $\texttt{person}$ attributes are affected by empty values. These will be handled the same way as the attributes above.

In [59]:
pe_values = ['_100', '_245c', '_700']

for pe in pe_values :
    df_feature_base = dpf.mark_missing(df_feature_base, 'person'+pe, factor)

### pubinit

This attribute holds publisher strings that have a similar representation as attribute $\texttt{person}$. A Jaro metric will be used.

In [60]:
attribute = 'pubinit'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Jaro()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [61]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

pubinit values range [0.         0.25132275 0.25303644 ... 0.9957265  0.99578059 1.        ]


In [62]:
position = uniques_len[attribute]//3

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position-1],
    uniques[attribute][uniques_len[attribute]-position], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-5],
    uniques[attribute][uniques_len[attribute]-1], 10)

,duplicates,pubinit_delta,pubinit_x,pubinit_y
38667,0,0.573477,universitätsverlag,[servizio topografico federale]
92630,0,0.573477,breitkopf & härtel,[servizio topografico federale]
34516,0,0.573477,ed. universitaires,[servizio topografico federale]
6645,0,0.573477,universitätsverlag,[servizio topografico federale]
51713,0,0.573477,universitätsverlag,[servizio topografico federale]
240507,0,0.573427,le grand livre du mois,teledec record service
126431,0,0.573477,ed. universitaires,[servizio topografico federale]
89665,0,0.573477,universitätsverlag,[servizio topografico federale]
236356,0,0.573427,le grand livre du mois,teledec record service


0.5734265734265734 <= pubinit_delta <= 0.5734767025089605


,duplicates,pubinit_delta,pubinit_x,pubinit_y
17624,0,1.0,,
59149,0,1.0,,
269245,1,1.0,,
270989,1,1.0,,
269611,1,1.0,,
60314,0,1.0,,
35981,0,1.0,,
80248,0,1.0,,
39443,0,1.0,deutsche grammophon,deutsche grammophon
272554,1,1.0,,


0.9952380952380953 <= pubinit_delta <= 1.0


The similarities of $\texttt{pubinit}$ is affected by empty values. These will be transformed to negative values.

In [63]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

### scale

Comparing the similarity metrics of some sample value pairs of attribute $\texttt{scale}$ in appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb), a Jaccard metrics has been identifyed to express the best matching behaviour for purely numerical values stored in the attribute.

In [64]:
attribute = 'scale'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Jaccard()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [65]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

scale values range [0.         0.17857143 0.21428571 0.57142857 1.        ]


In [66]:
position = uniques_len[attribute]

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-3],
    uniques[attribute][uniques_len[attribute]-2], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-4],
    uniques[attribute][uniques_len[attribute]-3], 10)

,duplicates,scale_delta,scale_x,scale_y
171483,0,0.0,,100000
178876,0,0.0,,100000
197265,0,0.0,100000,
84912,0,0.0,,100000
34506,0,0.0,,50000
19080,0,0.0,,100000
166749,0,0.0,,100000
70358,0,0.0,,100000
45779,0,0.0,,50000
3091,0,0.0,,100000


0.0 <= scale_delta <= 0.1785714285714286


,duplicates,scale_delta,scale_x,scale_y
227863,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
96182,0,0.571429,50000,100000
227821,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
55271,0,0.571429,50000,100000
227814,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
55254,0,0.571429,50000,100000
95852,0,0.571429,50000,100000
55277,0,0.571429,50000,100000
257524,0,0.571429,100000,50000
182432,0,0.571429,50000,100000


0.2142857142857143 <= scale_delta <= 0.5714285714285714


,duplicates,scale_delta,scale_x,scale_y
227832,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
227822,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
227842,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
227494,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
227831,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
227815,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
227501,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
227825,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
872,1,0.178571,50 000 8 10 8 35 45 55 46 05,50000
227845,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000


0.1785714285714286 <= scale_delta <= 0.2142857142857143


Attribute $\texttt{scale}$ is filled for maps, only. Due to its sparse filling, the similarities of the attribute are affected strongly by empty values. These empty values will be marked with a special negative value.

In [67]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

### ttlfull

Due to the discussion in chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{ttlfull}$ has been split up into two new attributes $\texttt{ttlfull_245}$ and $\texttt{ttlfull_246}$ which will be compared by the same similarity metrics. A visual analysis of the values stored in the attribute, reveals a string of words, comparable to the strings in attribute $\texttt{person_245c}$, above. The same similarity metric will be used for both title attributes, therefore.

In [68]:
attribute = 'ttlfull'

columns_metadata_dict['similarity_metrics'][attribute+'_245'] = tedi.Jaro()
columns_metadata_dict['similarity_metrics'][attribute+'_246'] = tedi.Jaro()

tf_values = ['_245', '_246']

for tf in tf_values :
    df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
        df_feature_base, attribute+tf,
        columns_metadata_dict['similarity_metrics'][attribute+tf],
        columns_metadata_dict)

In [69]:
for tf in tf_values :
    uniques[attribute+tf], uniques_len[attribute+tf] = dpf.determine_similarity_values(
        df_feature_base, attribute+tf)

ttlfull_245 values range [0.         0.24175824 0.24358974 ... 0.99947257 0.99947341 1.        ]
ttlfull_246 values range [0.         0.27407407 0.35555556 ... 0.99943978 0.99945085 1.        ]


In [70]:
tf = '_245'
position = uniques_len[attribute+tf]

dpf.show_samples_interval(
    df_feature_base, attribute+tf,
    uniques[attribute+tf][uniques_len[attribute+tf]-position],
    uniques[attribute+tf][uniques_len[attribute+tf]-position+1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute+tf,
    uniques[attribute+tf][uniques_len[attribute+tf]-3],
    uniques[attribute+tf][uniques_len[attribute+tf]-2], 10)
dpf.show_samples_interval(
    df_feature_base, attribute+tf,
    uniques[attribute+tf][uniques_len[attribute+tf]-4],
    uniques[attribute+tf][uniques_len[attribute+tf]-3], 10)

,duplicates,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y
96758,0,0.0,emma,blick in die welt
66514,0,0.0,emma,blick in die welt
96710,0,0.0,emma,blick in die welt
33849,0,0.0,arts,blick in die welt
11924,0,0.0,emma,blick in die welt
37672,0,0.0,bildungsforschung und bildungspraxis,emma
33953,0,0.0,arts,blick in die welt
13110,0,0.0,emma,blick in die welt
25806,0,0.0,bildungsforschung und bildungspraxis,emma
3914,0,0.0,emma,blick in die welt


0.0 <= ttlfull_245_delta <= 0.2417582417582418


,duplicates,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y
272725,1,0.999473,health informatics - personal health device co...,health informatics - personal health device co...
272723,1,0.999473,health informatics - personal health device co...,health informatics - persnoal health device co...
272715,1,0.999473,health informatics - personal health device co...,health informatics - personal health device co...
272709,1,0.999473,health informatics - personal health device co...,health informatics - personal health device co...


0.9994725738396625 <= ttlfull_245_delta <= 0.9994734070563455


,duplicates,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y
272602,1,0.999463,health informatics - personal health device co...,health informatics - personal health device co...
272723,1,0.999473,health informatics - personal health device co...,health informatics - persnoal health device co...
272715,1,0.999473,health informatics - personal health device co...,health informatics - personal health device co...


0.9994632313472893 <= ttlfull_245_delta <= 0.9994725738396625


Attribute $\texttt{ttlfull}\_\texttt{245}$ is filled for all data rows of Swissbib's raw data as can be seen in chapter [Data Analysis](./1_DataAnalysis.ipynb). For attribute $\texttt{ttlfull}\_\texttt{245}$, the filling is below $10\%$. The data pairs with missing values will be marked with a negative value as has been done for similar cases above.

In [71]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute+'_246', factor)

### volumes

This attribute is described in chapter [Data Analysis](./1_DataAnalysis.ipynb) to hold a kind of contents that resembles the contents of attribute $\texttt{part}$. The same similarity metrics will be used for attribute $\texttt{volumes}$ as for attribute $\texttt{part}$, therefore.

In [72]:
attribute = 'volumes'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.StrCmp95()
#tedi.Jaro()
#tedi.LCSSeq()
#tedi.MongeElkan()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [73]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

volumes values range [0.         0.31318681 0.32222222 0.32905983 0.35128205 0.37301587
 0.37407407 0.38333333 0.38461538 0.3952381  0.40740741 0.41111111
 0.41666667 0.41737892 0.42857143 0.43650794 0.43703704 0.44017094
 0.44166667 0.44200244 0.44230769 0.44761905 0.4537037  0.45555556
 0.45833333 0.46296296 0.46428571 0.46581197 0.46666667 0.47008547
 0.47222222 0.47619048 0.47777778 0.48148148 0.48333333 0.48412698
 0.48611111 0.48888889 0.49007937 0.49145299 0.49206349 0.49365079
 0.4991453  0.5        0.50793651 0.51111111 0.51190476 0.52222222
 0.52380952 0.52564103 0.52777778 0.53333333 0.53653846 0.53703704
 0.53968254 0.54074074 0.54166667 0.54304029 0.54401709 0.54722222
 0.54761905 0.5491453  0.55       0.55128205 0.55555556 0.56031746
 0.56111111 0.56190476 0.56507937 0.56630037 0.56666667 0.56944444
 0.57407407 0.57478632 0.57777778 0.58148148 0.58333333 0.58730159
 0.58862434 0.58888889 0.59444444 0.5952381  0.59722222 0.5982906
 0.6        0.60119048 0.60320513 0.606837

In [74]:
position = uniques_len[attribute]

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-3],
    uniques[attribute][uniques_len[attribute]-2], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-4],
    uniques[attribute][uniques_len[attribute]-3], 10)

,duplicates,volumes_delta,volumes_x,volumes_y
215195,0,0.0,407,192
177204,0,0.0,,15
187717,0,0.0,,471
111532,0,0.0,312,
176513,0,0.0,1,
235106,0,0.0,2,1 425
47282,0,0.0,64,101
6533,0,0.0,,3 33
234081,0,0.0,239,1
239923,0,0.0,72,450


0.0 <= volumes_delta <= 0.3131868131868132


,duplicates,volumes_delta,volumes_x,volumes_y
150878,0,0.933333,1 33,1 393
247561,0,0.933333,1 45,1 245
158546,0,0.916667,1 82,1 2
247197,0,0.916667,1 45,145
135609,0,0.916667,191,1 91
86200,0,0.916667,1 82,1 2
247551,0,0.933333,1 45,1 450
256983,0,0.916667,1 82,1 2
247085,0,0.933333,1 45,1 145
53581,0,0.933333,1 379,1 39


0.9166666666666666 <= volumes_delta <= 0.9333333333333332


,duplicates,volumes_delta,volumes_x,volumes_y
250460,0,0.916667,1 32,1 2
10889,0,0.916667,1 82,1 2
23934,0,0.916667,1 82,1 2
165069,0,0.916667,1 82,1 2
2114,0,0.904762,1 0 169,1 169
250334,0,0.916667,1 32,1 23
11,1,0.904762,1 0 169,1 169
86197,0,0.916667,1 82,1 2
86199,0,0.916667,1 82,1 2
165068,0,0.916667,1 82,1 2


0.9047619047619048 <= volumes_delta <= 0.9166666666666666


Attribute $\texttt{volumes}$ holds rows with missing data. The data pairs with missing values will be marked with a special negative value.

In [75]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

## DataFrame with Attributes and Similarity Features

The metric for each attribute of the feature DataFrame has been decided and the similarity features have been calculated. In this last step, the columns of the DataFrame are reordered in order to place the $\_\texttt{delta}$ columns close to their input origins $\_\texttt{x}$ and $\_\texttt{y}$ and some sample records are shown.

In [76]:
# Take _x, _y, and _delta columns together
fb_col_list = df_feature_base.columns.tolist()
fb_col_list.sort()
# Move target column to first place
fb_col_list.insert(0, fb_col_list.pop(fb_col_list.index('duplicates')))
# Reorder DataFrame columns
df_attribute_with_sim_feature = pd.DataFrame(df_feature_base, columns=fb_col_list)

# Extend display to number of columns of DataFrame
pd.options.display.max_columns = len(df_attribute_with_sim_feature.columns)

class_label = ['uniques', 'duplicate']

for i in class_label:
    display(df_attribute_with_sim_feature[df_attribute_with_sim_feature.duplicates==class_label.index(i)].sample(n=10))
    print(i)

,duplicates,coordinate_E_delta,coordinate_E_x,coordinate_E_y,coordinate_N_delta,coordinate_N_x,coordinate_N_y,corporate_full_delta,corporate_full_x,corporate_full_y,doi_delta,doi_x,doi_y,edition_delta,edition_x,edition_y,exactDate_delta,exactDate_x,exactDate_y,format_postfix_delta,format_postfix_x,format_postfix_y,format_prefix_delta,format_prefix_x,format_prefix_y,isbn_delta,isbn_x,isbn_y,ismn_delta,ismn_x,ismn_y,musicid_delta,musicid_x,musicid_y,part_delta,part_x,part_y,person_100_delta,person_100_x,person_100_y,person_245c_delta,person_245c_x,person_245c_y,person_700_delta,person_700_x,person_700_y,pubinit_delta,pubinit_x,pubinit_y,scale_delta,scale_x,scale_y,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y,ttlfull_246_delta,ttlfull_246_x,ttlfull_246_y,volumes_delta,volumes_x,volumes_y
40929,0,-1.0,,,-1.0,,,-1.0,,,-1.0,,,-1.000000,,,0.5000,aaaaaaaa,1994uuuu,0.428571,020000,030000,0.0,bk,vm,1.0,[],[],-1.0,,,-1.0,,,-1.0,,,-0.5,mozartwolfgang amadeus,,0.579105,von emanuel schikaneder ; musik von w. a. mozart,sigrid kessler... [et al.] ; [hrsg.:] interkan...,-0.500000,schikanederemanuel,,0.434788,bechtold,"interkantonale lehrmittelzentrale, staatlicher...",-1.0,,,0.637602,"die zauberflöte, grosse oper in 2 akten","bonne chance!, cours de langue française, 1",-1.0,,,0.000000,26,147
106336,0,-1.0,,,-1.0,,,-1.0,,,-1.0,,,-1.000000,,,0.6250,2011aaaa,20121970,0.428571,010300,010000,0.0,vm,mu,1.0,[],[],-0.5,,9790006201334,-0.5,,155,-0.5,1,,-0.5,,mozartwolfgang amadeus,0.481927,[regie:] volker schlöndorff,w. a. mozart ; herausgegeben von/edited by ger...,0.566387,"schlöndorffvolker, frischmax","schikanederemanuel, grubergernot, orelalfred, ...",-0.500000,suhrkamp,,-1.0,,,0.438141,"homo faber, nach dem roman von max frisch","die zauberflöte, kv 620",-1.0,,,0.733333,1 117,1 370
248503,0,-1.0,,,-1.0,,,-1.0,,,-1.0,,,0.166667,10425,11,0.2500,1932aaaa,2005uuuu,1.000000,010200,010200,1.0,mu,mu,1.0,[],[],-0.5,,m006450510,-0.5,,4553,-1.0,,,1.0,mozartwolfgang amadeus,mozartwolfgang amadeus,0.541118,w.a: mozart / [ hrsg. von kurt soldan],wolfgang amadeus mozart ; libretto: emanuel sc...,0.582222,"soldankurt, mozartwolfgang amadeus","schikanederemanuel, moehnheinz",-0.500000,c.f. peters,,-1.0,,,0.690191,"die zauberflöte, oper in 2 aufzügen","die zauberflöte, eine deutsche oper in zwei au...",-1.0,,,0.600000,1 188,1 221
206006,0,-1.0,,,-1.0,,,-0.5,schweizerische normen-vereinigung,,-1.0,,,-1.000000,,,0.2500,2011aaaa,1997uuuu,1.000000,020053,020053,1.0,bk,bk,0.0,[],"[978-3-598-31510-7 (print), 978-3-11-096990-0]",-1.0,,,-1.0,,,-0.5,,30 30,-0.5,,mortzfeldpeter,-0.500000,,"mortzfeld, peter; raabe, paul",-0.500000,,raabepaul,0.412821,schweizerische normen-vereinigung (snv),de gruyter saur,-1.0,,,0.574692,informatique de santé - communication entre di...,katalog der graphischen porträts in der herzog...,-0.5,medizinische informatik - kommunikation von ge...,,-0.500000,,1 385
97712,0,-1.0,,,-1.0,,,-0.5,opernhaus,,-1.0,,,-1.000000,,,0.2500,2002aaaa,1955uuuu,0.428571,010300,010200,0.0,vm,mu,1.0,[],[],-1.0,,,-0.5,,10425,-1.0,,,1.0,mozartwolfgang amadeus,mozartwolfgang amadeus,0.609434,wolfgang amadeus mozart ; libretto by emanuel ...,[musik] von w. a. mozart ; klavierauszug nach ...,0.692113,"schikanederemanuel, salminenmatti, scharingera...","zallingermeinhard von, schikanederemanuel",0.508258,"tdk recording media europe, opernhaus",peters,-1.0,,,0.752021,"die zauberflöte, the magic flute : opera in tw...","die zauberflöte, oper in 2 aufzügen",-1.0,,,0.000000,2 151,1
228006,0,-1.0,,,-1.0,,,-0.5,interkantonale lehrmittelzentrale (luzern),,-1.0,,,-1.000000,,,0.2500,19829999,2008uuuu,0.428571,030000,020000,0.0,vm,bk,0.0,[],[3-15-002620-2],-1.0,,,-1.0,,,-0.5,,2620,-0.5,,mozartwolfgang amadeus,0.636394,sigrid kessler... [et al.] ; [éd.:] interkanto...,wolfgang amadeus mozart ; libretto von emanuel...,0.558385,kesslersigrid,"kochhans-albrecht, schikanederemanuel",0.626984,staatlicher lehrmittelverlag,reclam,-1.0,,,0.627476,"bonne chance!, cours de langue française,

uniques


,duplicates,coordinate_E_delta,coordinate_E_x,coordinate_E_y,coordinate_N_delta,coordinate_N_x,coordinate_N_y,corporate_full_delta,corporate_full_x,corporate_full_y,doi_delta,doi_x,doi_y,edition_delta,edition_x,edition_y,exactDate_delta,exactDate_x,exactDate_y,format_postfix_delta,format_postfix_x,format_postfix_y,format_prefix_delta,format_prefix_x,format_prefix_y,isbn_delta,isbn_x,isbn_y,ismn_delta,ismn_x,ismn_y,musicid_delta,musicid_x,musicid_y,part_delta,part_x,part_y,person_100_delta,person_100_x,person_100_y,person_245c_delta,person_245c_x,person_245c_y,person_700_delta,person_700_x,person_700_y,pubinit_delta,pubinit_x,pubinit_y,scale_delta,scale_x,scale_y,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y,ttlfull_246_delta,ttlfull_246_x,ttlfull_246_y,volumes_delta,volumes_x,volumes_y
268300,1,-1.000,,,-1.000,,,-1.000000,,,-1.0,,,-1.0,,,0.8750,18761920,98761920,1.000000,030653,030653,1.0,cr,cr,1.0,[1013-0640],[1013-0640],-1.0,,,-1.0,,,-1.0,,,-1.0,,,1.000000,hrsg. auf veranstaltung der allgemeinen geschi...,hrsg. auf veranstaltung der allgemeinen geschi...,-1.000000,,,-1.000000,,,-1.0,,,0.991228,jahrbuch für schweizerische geschichte,jahrbuch fr schweizerische geschichte,-1.000000,,,-1.0,,
268654,1,-1.000,,,-1.000,,,0.983333,"interkantonale lehrmittelzentrale (rapperswil,...","nterkantonale lehrmittelzentrale (rapperswil, ...",-1.0,,,-1.0,,,1.0000,20002005,20002005,1.000000,020000,020000,1.0,bk,bk,1.0,[],[],-1.0,,,-1.0,,,-1.0,,,-1.0,,,0.996700,"sigrid kessler, brigitte suter, rudolf walther...","sigrid kessler, brigitte suter, rudolf walther...",1.000000,kesslersigrid,kesslersigrid,0.980672,berner lehrmittel- und medienverl.,berner lehrmittel- und cmedienver.l,-1.0,,,0.992248,"bonne chance !, cours de langue française 3","bonne chanc e!, cours de langue française 3",-1.000000,,,1.0,3,3
272640,1,-1.000,,,-1.000,,,0.727273,schweizerische normen-vereinigung,schweizerische normen-veeinigung,-1.0,,,-1.0,,,0.7500,2013aaaa,2013uuuu,1.000000,020053,020053,1.0,bk,bk,1.0,[],[],-1.0,,,-1.0,,,-1.0,,,-1.0,,,-1.000000,,,-1.000000,,,-1.000000,,,-1.0,,,0.890512,health informatics - personal health device co...,health informatics - personal health device co...,0.996816,informatique de santé - communication entre di...,informatique de santé - communication entre di...,1.0,1,1
269359,1,-1.000,,,-1.000,,,-1.000000,,,-1.0,,,-1.0,,,0.5000,aaaaaaaa,uuuuuu8u,1.000000,010000,010000,1.0,mu,mu,1.0,[],[],-1.0,,,1.0,140,140,-1.0,,,1.0,mozartwolfgang amadeus,mozartwolfgang amadeus,0.944444,w. a. mozart,w. ai mozart,0.981818,mozartwolfgang amadeus,mozartwotfgang amadeus,0.921296,breitkopf,brektopf,-1.0,,,0.957479,"die zauberflöte, [extraits pour choeur]","die zauberflötei,[ extraits pour choeur]",-1.000000,,,1.0,1 65,1 65
1014,1,-0.500,e0060811,,-0.500,n0461242,,-1.000000,,,-1.0,,,-1.0,,,0.6875,17001799,170uuuuu,0.428571,010700,010300,1.0,mp,mp,1.0,[],[],-1.0,,,-1.0,,,-1.0,,,-1.0,,,-1.000000,,,-0.500000,,fernicolas de,-0.500000,,de fer,-1.0,,,0.513054,"genéve, genff","genève, ville capitale d'une république de mêm...",-1.000000,,,1.0,1,1
266448,1,-1.000,,,-1.000,,,-1.000000,,,1.0,10.5169/seals-376994,10.5169/seals-376994,-1.0,,,0.7500,1997aaaa,1997uuuu,1.000000,010053,010053,1.0,bk,bk,1.0,[],[],-1.0,,,-1.0,,,1.0,276 1997,276 1997,1.0,bührerwalter,bührerwalter,0.979167,[walter bührer],[waltery bührer],-1.000000,,,-1.000000,,,-1.0,,,1.000000,blick in die welt,blick in die welt,-1.000000,,,-1.0,,
272448,1,-1.000,,,-1.000,,,-1.000000,,,-1.0,,,-1.0,,,0.7500,1979aaaa,1979uuuu,0.666667,020000,200000,1.0,bk,bk,1.0,[],[],-1.0,,,-1.0,,,-1.0,,,1.0,austenjane,austenjane,0.969697,jane austen,jane asten,-1.000000,,,-1.000000,,,-1.0,,,0.916667,emma,emam,-1.000000,,,1.0,549,549
267366,1,0.125,e0074147,n0460833,0.375,n0460833,n0406833,0.875000,eidgenössische landestopographie,eidgenössische landestopograhpie,-1.0,,,-1.0,,,0.7500,1943aaaa,1943uuuu,1.000000,010300,010300,1.0,mp,mp,1.0,[],[],-1.0,,,-1.0,,,1.0,23 1943 23 1943 23,23 1943 23 1943 23,-1.0,,,0.985017,g. h. dufour d

duplicate


## Summary

This chapter covers the central area of feature construction. The features of the feature matrix have been generated for each attribute of Swissbib's raw data, deciding on its similarity metric. With these metric values, the feature base DataFrame has been extended and a new DataFrame with the attribute values of the pairs together with their calculated similarity value have been generated. The similarity values will be the final features for training and performance testing of the models, compare [[JudACaps](./A_References.ipynb#judacaps)].

In [77]:
columns_metadata_dict['similarity_metrics']

{'coordinate_E': LCSStr({'qval': 1, 'external': True}),
 'coordinate_N': LCSStr({'qval': 1, 'external': True}),
 'corporate_full': LCSStr({'qval': 1, 'external': True}),
 'doi': Identity({'qval': 1, 'external': True}),
 'edition': Jaccard({'qval': 1, 'as_set': False, 'external': True}),
 'exactDate': Hamming({'qval': 1, 'test_func': <function Base._ident at 0x11e28b2f0>, 'truncate': False, 'external': True}),
 'format_prefix': Identity({'qval': 1, 'external': True}),
 'format_postfix': Jaccard({'qval': 2, 'as_set': False, 'external': True}),
 'isbn': Identity({'qval': 1, 'external': True}),
 'ismn': Identity({'qval': 1, 'external': True}),
 'musicid': LCSStr({'qval': 1, 'external': True}),
 'part': StrCmp95({'long_strings': False, 'external': True}),
 'person_100': StrCmp95({'long_strings': False, 'external': True}),
 'person_700': StrCmp95({'long_strings': False, 'external': True}),
 'person_245c': Jaro({'qval': 1, 'long_tolerance': False, 'winklerize': False, 'external': True}),
 'pu

The similarity metric decided for each attribute has been added as an additional kind of information to the columns metadata dictionary. The following table gives this summary in a structured form and lists the metric used for each attribute. Attributes with the same font color indicate similar types of values (description column) for better orientation.

| attribute     | subtype | description | similarity metric |
| ------------- |:--------|:------------|:------------------|
|<font color='red'>[coordinate](#coordinate)</font>|<font color='red'>\_E</font>|<font color='red'>Code(9)</font>|<font color='red'>LCSStr</font>|
|               |<font color='red'>\_N</font>|<font color='red'>Code(9)</font>|<font color='red'>LCSStr</font>|
|<font color='blue'>[corporate](#corporate)</font>|<font color='blue'>\_full</font>|<font color='blue'>Name</font>|<font color='blue'>LCSStr</font>|
|<font color='green'>[doi](#doi)</font>|         |<font color='green'>Identifier</font>|<font color='green'>Identity</font>|
|<font color='orange'>[edition](#edition)</font>|         |<font color='orange'>Number</font>|<font color='orange'>Jaccard</font>|
|<font color='black'>[exactDate](#exactDate)</font>|         |<font color='black'>Date</font>|<font color='black'>Hamming</font>|
|<font color='red'>[format](#format)</font>|<font color='red'>\_prefix</font>|<font color='red'>Code(2)</font>|<font color='red'>Identity</font>|
|               |<font color='red'>\_postfix</font>|<font color='red'>Code(6)</font>|<font color='red'>Jaccard (qval=2)</font>|
|<font color='green'>[isbn](#isbn)</font>|         |<font color='green'>Identifier</font>|<font color='green'>Identity</font>|
|<font color='green'>[ismn](#ismn)</font>|         |<font color='green'>Identifier</font>|<font color='green'>Identity</font>|
|<font color='green'>[musicid](#musicid)</font>|         |<font color='green'>Identifier</font>|<font color='green'>LCSStr</font>|
|<font color='orange'>[part](#part)</font>|         |<font color='orange'>Number</font>|<font color='orange'>StrCmp95</font>|
|<font color='blue'>[person](#person)</font>|<font color='blue'>\_100</font>|<font color='blue'>Name</font>|<font color='blue'>StrCmp95</font>|
|               |<font color='blue'>\_700</font>|<font color='blue'>Name</font>|<font color='blue'>StrCmp95</font>|
|               |<font color='blue'>\_245c</font>|<font color='blue'>Name</font>|<font color='blue'>Jaro</font>|
|<font color='blue'>[pubinit](#pubinit)</font>|         |<font color='blue'>Name</font>|<font color='blue'>Jaro</font>|
|<font color='orange'>[scale](#scale)</font>|         |<font color='orange'>Number</font>|<font color='orange'>Jaccard</font>|
|<font color='blue'>[ttlfull](#ttlfull)</font>|<font color='blue'>\_245</font>|<font color='blue'>String</font>|<font color='blue'>Jaro</font>|
|               |<font color='blue'>\_246</font>|<font color='blue'>String</font>|<font color='blue'>Jaro</font>|
|<font color='orange'>[volumes](#volumes)</font>|         |<font color='orange'>Number</font>|<font color='orange'>StrCmp95</font>|

### Full Feature Matrix with Target Vector Handover

To hand over the resulting DataFrame of this chapter, the DataFrame is saved into a pickle file that will be read in the next chapters [Features Discussion and Dummy Classifier Baseline](./5_FeatureDiscussionDummyBaseline.ipynb) as input.

In [78]:
# Store into compressed intermediary file
with bz2.BZ2File(os.path.join(path_goldstandard,
                       'labelled_feature_matrix_full.pkl'), 'w') as df_output_file:
    pk.dump(df_attribute_with_sim_feature, df_output_file)
    
# Binary intermediary DataFrame file for docid's
with open(os.path.join(path_goldstandard, 'index_docids_df.pkl'), 'wb') as df_output_file:
    pk.dump(df_index_docids, df_output_file)

The full metadata dictionary is to be persisted for handover to subsequent chapters.

In [79]:
# The target is still needed for the feature matrix
columns_metadata_dict['features'].append('duplicates')

for k in columns_metadata_dict.keys():
    print(k, '\n', columns_metadata_dict[k], '\n')

data_analysis_columns 
 ['coordinate_E', 'coordinate_N', 'corporate_full', 'doi', 'edition', 'exactDate', 'format_prefix', 'format_postfix', 'isbn', 'ismn', 'musicid', 'part', 'person_100', 'person_700', 'person_245c', 'pubinit', 'scale', 'ttlfull_245', 'ttlfull_246', 'volumes'] 

columns_to_use 
 ['duplicates', 'coordinate_E_x', 'coordinate_E_y', 'coordinate_N_x', 'coordinate_N_y', 'corporate_full_x', 'corporate_full_y', 'doi_x', 'doi_y', 'edition_x', 'edition_y', 'exactDate_x', 'exactDate_y', 'format_prefix_x', 'format_prefix_y', 'format_postfix_x', 'format_postfix_y', 'isbn_x', 'isbn_y', 'ismn_x', 'ismn_y', 'musicid_x', 'musicid_y', 'part_x', 'part_y', 'person_100_x', 'person_100_y', 'person_700_x', 'person_700_y', 'person_245c_x', 'person_245c_y', 'pubinit_x', 'pubinit_y', 'scale_x', 'scale_y', 'ttlfull_245_x', 'ttlfull_245_y', 'ttlfull_246_x', 'ttlfull_246_y', 'volumes_x', 'volumes_y'] 

similarity_metrics 
 {'coordinate_E': LCSStr({'qval': 1, 'external': True}), 'coordinate_N': L

In [80]:
# Binary intermediary metadata file
with open(os.path.join(path_goldstandard,
                       'columns_metadata.pkl'), 'wb') as dict_output_file:
    pk.dump(columns_metadata_dict, dict_output_file)